In [ ]:
# 目标 熟悉 langchain 调用大模型
# 1.在 LangChain 中使用 LLM
# 2.使用 prompt templates 组件
# 3.使用 LangSmith 调试和追踪

In [ ]:
import sys

from jupyterlab.extensions.pypi import http_proxy
from langchain.chat_models import init_chat_model
from langchain_community.llms.openai import OpenAI
# 安装依赖
%pip install langchain

In [ ]:
# 相关环境变量设置
import config_loader

config_loader.load_env()

In [ ]:
# 安装谷歌聊天模型 可以根据自己的情况自行选择
%pip install -U langchain-google-genai

In [ ]:
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.messages import SystemMessage, HumanMessage

# 配置谷歌聊天模型
model = ChatGoogleGenerativeAI(
    model="gemini-2.0-flash",
    temperature=0,
    max_tokens=None,
    timeout=None,
    max_retries=2,
    # other params...
)

# SystemMessage: 启动 AI 行为的信息，作为消息序列的第一个传入输入消息
# HumanMessage: 用户传入的消息
messages = [
    SystemMessage("把下面的中文翻译成英文"),
    HumanMessage("你是谁？"),
]
ai_msg = model.invoke(messages)
print(ai_msg)
print(ai_msg.content)

In [ ]:
# 发送消息调用模型 下面三种方式是等价的
model.invoke("Hello")

model.invoke([{"role": "user", "content": "Hello"}])

model.invoke([HumanMessage("Hello")])

In [ ]:
# 消息流式传输 下面将会一个token一个token的输出，并且不同token之间以 | 分隔
for token in model.stream(messages):
    print(token.content, end="|")

In [ ]:
# 提示词模板
# 定义一套可复用的提示词模板，可以用提示词模板结构化提示词，将提示词分为系统指令和用户输入两部分，易于理解和维护，并且可传参，动态替换提示词模板中的变量
# 可复用性、结构化提示词、灵活性
from langchain_core.prompts import ChatPromptTemplate

system_template = "把这段文本翻译成{language}"
# 定义系统角色和用户角色各自的发言
prompt_template = ChatPromptTemplate.from_messages(
    [("system", system_template), ("user", "{text}")]
)
# 在这里传参
prompt = prompt_template.invoke({
    "language": "英文",
    "text": "你好呀",
})
print(prompt)
print(prompt.to_messages())
# 测试模型调用
ai_msg = model.invoke(prompt)
ai_msg.content